# Mock interview Seite für Studenten

## Für dieses Interview wirst du mit dem folgenden Datensatz arbeiten. Schau ihn dir gerne in Ruhe an, um dich damit vertraut zu machen. 
- loans_modified.csv

 ## Ziel des Interviews:
 - Erstelle ein Modell zur Analyse von Darlehen.

# Schritte zur Vorbereitung:
- 1. Mach dich mit den Daten vertraut.
- 2. Versuch, ein Modell zu erstellen.
- 3. Wir empfehlen dir, ein Notebook bereitzuhalten, um während des Interviews Notizen zu machen.

In [ ]:
# ⚙️ Laden von CSV-Dateien in Colab/VSCode-Umgebung
# ============================================================
import pandas as pd
import os
import warnings
from tkinter import filedialog
from tkinter import Tk

# Einstellungen
warnings.filterwarnings("ignore")

# Erkennung der Umgebung (Google Colab oder lokal)
try:
    from google.colab import files
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

# ====== DATEI LADEN ======
print('*' * 10, 'DATEI LADEN', '*' * 10)

try:
    if COLAB_ENV:
        # Code für Google Colab
        print("Google Colab Umgebung erkannt. Bitte Datei hochladen.")
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        print(f"\nDie Datei '{file_name}' wurde erfolgreich hochgeladen und steht nun zur Verfügung.")
        df = pd.read_csv(file_name)
    else:
        # Code für lokale Umgebung mit Tkinter-Dateiauswahlfenster
        print("Lokale Umgebung erkannt. Es öffnet sich ein Dateiauswahlfenster.")
        
        # Erstellen eines Tkinter-Root-Fensters und Ausblenden
        root = Tk()
        root.withdraw() 
        
        # Öffnen des Dateidialogs und Abrufen des Pfades
        file_path = filedialog.askopenfilename(
            title="Wählen Sie Ihre CSV-Datei",
            filetypes=(("CSV-Dateien", "*.csv"), ("Alle Dateien", "*.*"))
        )
        
        if file_path:
            df = pd.read_csv(file_path)
            print(f"\nDie Datei '{file_path}' wurde erfolgreich geladen.")
        else:
            print("Keine Datei ausgewählt.")
            df = None
            
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")
    df = None

********** DATEI LADEN **********
Lokale Umgebung erkannt. Es öffnet sich ein Dateiauswahlfenster.

Die Datei '/Users/cristallagus/Desktop/GitHub/Mock Interviwe 07.09.2025/loans_modified.csv' wurde erfolgreich geladen.


In [5]:
# ⚙️ DataFrames als CSV-Datei in df benennen
# ============================================================
import pandas as pd
print('*' * 10, 'DATEN LADEN', '*' * 10)

df = pd.read_csv('loans_modified.csv')

# Erfolgsnachweis: Ausgabe der Dateninformationen
print(f"Daten erfolgreich geladen! DataFrame-Größe: {df.shape}")
print("\nErste 5 Zeilen des geladenen DataFrames zur Überprüfung:")
print(df.head().to_string())

print('*' * 50)

********** DATEN LADEN **********
Daten erfolgreich geladen! DataFrame-Größe: (563, 13)

Erste 5 Zeilen des geladenen DataFrames zur Überprüfung:
    loan_id gender married dependents     education self_employed  applicant_income  coapplicant_income  loan_amount  loan_amount_term  credit_history property_area  loan_status
0  LP001003   Male     Yes          1      Graduate            No            4583.0              1508.0        128.0             360.0             1.0         Rural          0.0
1  LP001005   Male     Yes          0      Graduate           NaN            3000.0                 0.0         66.0             360.0             1.0         Urban          1.0
2  LP001006   Male     Yes          0  Not Graduate            No            2583.0              2358.0        120.0             360.0             1.0         Urban          1.0
3  LP001008   Male      No          0      Graduate            No            6000.0                 0.0        141.0             360.0        

In [25]:
import pandas as pd
import numpy as np
import warnings
from typing import Dict, List, Callable, Union

# ----------------------------------------------------------------------
# HILFSFUNKTIONEN FÜR SEMANTISCHE MUSTER-ERKENNUNG (AUS IHREM VORSCHLAG)
# ----------------------------------------------------------------------

# NOTE: generate_cleaning_muster wird ignoriert, da keine Vorschläge gewünscht sind
def generate_cleaning_muster(column: str, semantic_type: str) -> str:
    # Nur ein Platzhalter, da keine Code-Ausgabe gewünscht ist
    return ""

def analyze_semantic_type_v3(df: pd.DataFrame) -> pd.DataFrame:
    """
    Analysiert die semantischen Datentypen der Spalten in einem DataFrame.
    Die interne Logik zur Typ-Erkennung wurde von Ihrem bereitgestellten Muster übernommen.
    """
    SEMANTIC_HINTS_PRIORITY: Dict[str, Dict[str, Union[set, Callable]]] = {
        'ID': {
            'keywords': {'_id', 'session_id', 'trip_id', 'user_id', 'unique_id', 'kundennummer', 'bestellnr', 'order_id', 'artikelnummer'},
            'validation_func': lambda series: ((series.dropna().astype(str).apply(len) >= 5).any())
        },
        'Datum/Zeit': {
            'keywords': {'datum', 'zeit', 'date', 'time', 'start', 'end', 'birthdate', 'signup_date', 'check_in', 'check_out', 'departure', 'return', 'geburtstag', 'timestamp', 'creation_date', 'modified_date', 'erstellt'},
            'validation_func':lambda series: ((series.dropna().nunique() == 12 or series.dropna().nunique() == 31) or (series.dropna().apply(lambda x: isinstance(x, str)).all() and (pd.to_datetime(series.dropna(), errors='coerce').notna().all() or (series.dropna().astype(str).str.contains(r'[-_/]', na=False).any() and series.dropna().astype(str).str.contains(r'\d{4}', na=False).any()))))
        },
        'Geometrisch': {
            'keywords': {'geom', 'geometry', 'shape', 'wkt', 'geojson', 'coordinates', 'location_data'},
            'validation_func': lambda series: (series.dropna().astype(str).str.contains(r'^(POINT|LINESTRING|POLYGON|MULTIPOINT|MULTILINESTRING|MULTIPOLYGON)\s*\(', regex=True, na=False).any() or series.dropna().astype(str).str.contains(r'{"type":\s*"(Point|LineString|Polygon|MultiPoint|MultiLineString|MultiPolygon)"', regex=True, na=False).any())
        },
    }
    SEMANTIC_HINTS_TEXT: Dict[str, Dict[str, Union[set, Callable]]] = {
        'Text (Kategorisch)': {
            'keywords': {'city', 'country', 'länder', 'region', 'state', 'bundesland', 'zip', 'plz'},
            'validation_func': lambda series: series.dropna().nunique() >= 2 and (pd.api.types.is_string_dtype(series.dropna()) or isinstance(series.dropna().dtype, pd.CategoricalDtype))
        },
         'Text (Gender)': {
            'keywords': {'geschlecht', 'typ', 'category', 'art', 'gender'},
            'validation_func': lambda series: series.dropna().nunique() >= 2 and (pd.api.types.is_string_dtype(series.dropna()) or isinstance(series.dropna().dtype, pd.CategoricalDtype))
        },
        'Text (object)': {
            'keywords': {'airport', 'destination', 'origin', 'heimat', 'status'},
            'validation_func': lambda series: series.dropna().nunique() >= 2 and (pd.api.types.is_string_dtype(series.dropna()) or isinstance(series.dropna().dtype, pd.CategoricalDtype))
        },
        'Text (Freitext)': {
            'keywords': {'name', 'hotel', 'airline', 'beschreibung', 'kommentar', 'nachricht', 'adresse'},
            'validation_func': lambda series: pd.api.types.is_string_dtype(series.dropna()) or isinstance(series.dropna().dtype, pd.CategoricalDtype)
        },
    }
    SEMANTIC_HINTS_NUMERIC: Dict[str, Dict[str, Union[set, Callable]]] = {
        'Boolean(NaN)': {
            'keywords': {'is_invalid','missing', 'is_missing', 'has_value', 'exists', 'is_null', 'is_na', 'isnan', 'filled','is_outlier'},
            'validation_func': lambda series: (series.dropna().nunique() >= 1) and (pd.api.types.is_bool_dtype(series.dropna()) or set(series.dropna().astype(str).str.lower().str.strip().unique()).issubset({'true', 'false', '1', '0', 'ja', 'nein', 'yes', 'no', 't', 'f', 'wahr', 'falsch'}))
        },
        'Boolean': {
            'keywords': {'self_employed','is_weekend_trip', 'boolean', 'bool', 'booked', 'married', 'cancellation', 'children','discount', 'flight_booked', 'hotel_booked', 'return_flight_booked', 'is_cancelled'},
            'validation_func': lambda series: (series.dropna().nunique() == 2) and (pd.api.types.is_bool_dtype(series.dropna()) or set(series.dropna().astype(str).str.lower().str.strip().unique()).issubset({'true', 'false', '1', '0', 'ja', 'nein', 'yes', 'no', 't', 'f', 'wahr', 'falsch'}))
        },
        'Float (Geografisch)': {
            'keywords': {'lat', 'lon', 'latitude', 'longitude'},
            'validation_func': lambda series: pd.to_numeric(series.dropna(), errors='coerce').notna().all() and (pd.to_numeric(series.dropna(), errors='coerce').astype(str).str.count(r'\.').all() or pd.api.types.is_float_dtype(series.dropna()))
        },
        'Float (Prozentsatz)': {
            'keywords': {'percent', 'pct', 'rate', 'discount', '%'},
            'validation_func': lambda series: (series.dropna().nunique() > 2) and ((pd.to_numeric(series.dropna().astype(str).str.replace('%', ''), errors='coerce').dropna().between(0, 1).all() or pd.to_numeric(series.dropna().astype(str).str.replace('%', ''), errors='coerce').dropna().between(0, 100).all()) or (pd.to_numeric(series.dropna().astype(str).str.replace('%', ''), errors='coerce').notna().all() and series.dropna().astype(str).str.replace('%', '').str.replace(',', '.').str.match(r'^\d{1,3}(\.\d{1,3})?$').all()))
        },
        'Float (Waehrung)': {
            'keywords': {'preis', 'kosten', 'betrag', 'dollar', 'euro', 'yen', 'usd', 'eur', 'fare','chf', 'gbp', 'sek', 'jpy', '€', '£', '$'},
            'validation_func': lambda series: (pd.api.types.is_numeric_dtype(series.dropna()) or pd.to_numeric(series.dropna().str.replace(',', '.'), errors='coerce').notna().all()) and series.dropna().nunique() > 2
        },
        'Integer': {
            'keywords': {'_time_days','_duration_days','anzahl', 'menge', 'stueck', 'stk', 'count', 'qty', 'seats', 'rooms', 'nights', 'bags', 'clicks', 'nummer', 'nr', 'quantity', 'val', 'rating'},
            'validation_func': lambda series: (series.dropna().nunique() > 2) and pd.to_numeric(series.dropna(), errors='coerce').notna().all() and (pd.to_numeric(series.dropna(), errors='coerce').dropna().apply(lambda x: x.is_integer() if isinstance(x, float) else True).all())
        }
    }

    results: List[Dict[str, str]] = []
    hint_categories = [SEMANTIC_HINTS_PRIORITY, SEMANTIC_HINTS_TEXT, SEMANTIC_HINTS_NUMERIC]
    SEMANTIC_HINTS_NUMERIC_ORDERED: List[str] = ['Boolean(NaN)','Boolean', 'Float (Geografisch)', 'Float (Prozentsatz)', 'Float (Waehrung)', 'Integer']

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", DeprecationWarning)
        warnings.simplefilter("ignore", UserWarning)

        for column in df.columns:
            original_dtype: str = str(df[column].dtype)
            semantic_type: str = original_dtype
            column_lower: str = column.lower()
            found_match: bool = False

            for hint_group in hint_categories:
                if found_match:
                    break
                if hint_group is SEMANTIC_HINTS_NUMERIC:
                    for sem_type in SEMANTIC_HINTS_NUMERIC_ORDERED:
                        hints = hint_group[sem_type]
                        name_match = any(keyword in column_lower for keyword in hints['keywords'])
                        content_valid = False
                        try:
                            content_valid = hints['validation_func'](df[column])
                        except Exception:
                            pass

                        if name_match and content_valid:
                            semantic_type = sem_type
                            found_match = True
                            break
                else:
                    for sem_type, hints in hint_group.items():
                        name_match = any(keyword in column_lower for keyword in hints['keywords'])
                        content_valid = False
                        try:
                            content_valid = hints['validation_func'](df[column])
                        except Exception:
                            pass

                        if name_match and content_valid:
                            semantic_type = sem_type
                            found_match = True
                            break

            results.append({
                'Spalte': column,
                'Semantischer Typ': semantic_type,
            })

    return pd.DataFrame(results)


# ----------------------------------------------------------------------
# HAUPT-MUSTER FÜR KONSOLIDIERTE ANZEIGE (INKL. SEMANTIK, OHNE VORSCHLÄGE)
# ----------------------------------------------------------------------

def muster_df_consolidated_view(df: pd.DataFrame) -> None:
    """
    Führt eine konsolidierte Datenqualitäts-Analyse durch, inklusive semantischer
    Typ-Erkennung, und gibt diese in einer einzigen Tabelle aus, ohne
    Bereinigungsvorschläge.
    """
    
    # 1. Semantische Typ-Erkennung
    df_sem_types = analyze_semantic_type_v3(df)
    
    # Dictionary zur Konsolidierung ALLER Metriken pro Spalte
    consolidated_data: Dict[str, Dict[str, Union[str, float, int, None]]] = {}

    # Initialisierung und Basis-Metriken
    for col in df.columns:
        sem_type_row = df_sem_types[df_sem_types['Spalte'] == col].iloc[0] if col in df_sem_types['Spalte'].values else {'Semantischer Typ': str(df[col].dtype)}
        
        consolidated_data[col] = {
            'Spalte': col,
            'Semantischer Typ': sem_type_row['Semantischer Typ'],
            'Ursprünglicher Datentyp': str(df[col].dtype),
            'Einzigartige Werte': df[col].nunique(),
            'Fehlende Werte (Anzahl)': df[col].isnull().sum(),
            'Fehlende Werte (%)': round(df[col].isnull().sum() / len(df) * 100, 2),
            
            # Statistische und ML-Metriken (Standardmäßig NaN)
            'Min': np.nan, '25% (Q1)': np.nan, 'Median': np.nan, '75% (Q3)': np.nan, 'Max': np.nan,
            'Skewness': np.nan,
            'Ausreißer (IQR-Anzahl)': 0,
        }

    # 2. Numerische Metriken und ML-Analyse
    # Spalten, die nach der semantischen Analyse als numerisch gelten könnten (inkl. Booleans)
    numeric_relevant_types = {'Float (Geografisch)', 'Float (Prozentsatz)', 'Float (Waehrung)', 'Integer', 'Boolean', 'Boolean(NaN)'}
    
    for _, row in df_sem_types.iterrows():
        column = row['Spalte']
        semantic_type = row['Semantischer Typ']
        series = df[column]

        # Führe numerische Analysen nur für als numerisch erkannte Spalten durch
        if semantic_type in numeric_relevant_types or pd.api.types.is_numeric_dtype(series):
            try:
                # Verwenden von to_numeric mit errors='coerce' ist sicherer für gemischte Typen
                numeric_series = pd.to_numeric(series, errors='coerce').dropna()
                
                if not numeric_series.empty:
                    q1, median, q3 = numeric_series.quantile([0.25, 0.5, 0.75])
                    min_val = numeric_series.min()
                    max_val = numeric_series.max()

                    skewness = round(numeric_series.skew(), 2)
                    
                    Q1_o, Q3_o = numeric_series.quantile([0.25, 0.75])
                    IQR = Q3_o - Q1_o
                    lower_bound = Q1_o - 1.5 * IQR
                    upper_bound = Q3_o + 1.5 * IQR
                    outliers_count = ((numeric_series < lower_bound) | (numeric_series > upper_bound)).sum()
                    
                    consolidated_data[column].update({
                        'Min': round(min_val, 2),
                        '25% (Q1)': round(q1, 2),
                        'Median': round(median, 2),
                        '75% (Q3)': round(q3, 2),
                        'Max': round(max_val, 2),
                        'Skewness': skewness,
                        'Ausreißer (IQR-Anzahl)': int(outliers_count)
                    })
            except Exception:
                pass # Bleibt bei NaN, falls die Umwandlung fehlschlägt

    # 3. KONSOLIDIERTE AUSGABE
    
    final_df = pd.DataFrame(list(consolidated_data.values()))
    
    # Spaltenreihenfolge für die Lesbarkeit anpassen
    column_order = [
        'Spalte', 'Semantischer Typ', 'Ursprünglicher Datentyp', 'Einzigartige Werte',
        'Fehlende Werte (Anzahl)', 'Fehlende Werte (%)', 
        'Min', '25% (Q1)', 'Median', '75% (Q3)', 'Max', 'Skewness',
        'Ausreißer (IQR-Anzahl)',
    ]
    final_df = final_df[[col for col in column_order if col in final_df.columns]]
    
    # NaN-Werte und 0-Werte für Nicht-Numerische durch "-" ersetzen
    for stat_col in ['Min', '25% (Q1)', 'Median', '75% (Q3)', 'Max', 'Skewness']:
        final_df[stat_col] = final_df[stat_col].apply(lambda x: '-' if pd.isna(x) else x)
        
    final_df['Ausreißer (IQR-Anzahl)'] = final_df.apply(
        lambda row: '-' if row['Semantischer Typ'] not in numeric_relevant_types and not pd.api.types.is_numeric_dtype(df[row['Spalte']]) else int(row['Ausreißer (IQR-Anzahl)']) if row['Ausreißer (IQR-Anzahl)'] is not None else 0,
        axis=1
    )


    # ====== FINALE AUSGABE ======
    print('*' * 10, 'KONSOLIDIERTE DATENQUALITÄTS-ANALYSE', '*' * 10)
    print(f"Form (Zeilen, Spalten): {df.shape} | Duplikate: {df.duplicated().sum()}")
    print("-" * 50)
    # Ausgabe der finalen, konsolidierten Tabelle
    print(final_df.to_string())
    print('*' * 50)
    # ============================

# HAUPTTEIL DES SKRIPTS
if 'df' in locals() and isinstance(df, pd.DataFrame):
    muster_df_consolidated_view(df)
else:
    print("Bitte laden Sie Ihren Datensatz in einen Pandas DataFrame namens 'df'!")

********** KONSOLIDIERTE DATENQUALITÄTS-ANALYSE **********
Form (Zeilen, Spalten): (563, 13) | Duplikate: 25
--------------------------------------------------
                Spalte Semantischer Typ Ursprünglicher Datentyp  Einzigartige Werte  Fehlende Werte (Anzahl)  Fehlende Werte (%)    Min 25% (Q1)  Median 75% (Q3)      Max Skewness Ausreißer (IQR-Anzahl)
0              loan_id               ID                  object                 512                       29                5.15      -        -       -        -        -        -                      -
1               gender    Text (Gender)                  object                   2                       29                5.15      -        -       -        -        -        -                      -
2              married          Boolean                  object                   2                       19                3.37      -        -       -        -        -        -                      0
3           dependents      